## Requirements
OpenAI API key
Libraries: openai, pandas, numpy, yfinance

## Setup
### First, install the necessary libraries:

pip install openai pandas numpy yfinance

### Step-by-Step Code
1. Import Necessary Libraries
2. Fetch Data
3. Preprocess Data
4. Generate Predictions using OpenAI API

### 1. Import Necessary Libraries

In [18]:
import openai
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

### 2. Fetch Data
We will use yfinance to fetch historical market data.

In [50]:
def fetch_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Example usage
ticker = 'AAPL'
start_date = '2023-06-01'
end_date = '2024-07-10'
data = fetch_data(ticker, start_date, end_date)
print(data.head())
print(data.tail())

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-06-01  177.699997  180.119995  176.929993  180.089996  179.138626   
2023-06-02  181.029999  181.779999  179.259995  180.949997  179.994095   
2023-06-05  182.630005  184.949997  178.039993  179.580002  178.631332   
2023-06-06  179.970001  180.119995  177.429993  179.210007  178.263306   
2023-06-07  178.440002  181.210007  177.320007  177.820007  176.880630   

               Volume  
Date                   
2023-06-01   68901800  
2023-06-02   61945900  
2023-06-05  121946500  
2023-06-06   64848400  
2023-06-07   61944600  
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-07-02  216.149994  220.380005  215.100006  220.270004  220.270004   
2024-07-03  220.000000  221.550003  219.029999  221.550003  221.550003   
2024-07-05  221.6

### 3. Preprocess Data
Calculate the indicators like moving average, RSI, etc.

In [51]:
def calculate_indicators(data):
    # Calculate Moving Averages
    data['MA50'] = data['Close'].rolling(window=50).mean()
    data['MA200'] = data['Close'].rolling(window=200).mean()

    # Calculate RSI
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    RS = gain / loss
    data['RSI'] = 100 - (100 / (1 + RS))

    # Drop NaN values
    data = data.dropna()
    
    return data

data = calculate_indicators(data)
print(data.tail())

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-07-02  216.149994  220.380005  215.100006  220.270004  220.270004   
2024-07-03  220.000000  221.550003  219.029999  221.550003  221.550003   
2024-07-05  221.649994  226.449997  221.649994  226.339996  226.339996   
2024-07-08  227.089996  227.850006  223.250000  227.820007  227.820007   
2024-07-09  227.929993  229.399994  226.369995  228.679993  228.679993   

              Volume      MA50      MA200        RSI  
Date                                                  
2024-07-02  58046200  192.7818  184.13120  65.641398  
2024-07-03  37369800  193.8960  184.36390  61.367286  
2024-07-05  60412400  195.0848  184.60575  64.784936  
2024-07-08  59085900  196.2608  184.84950  68.856087  
2024-07-09  48076100  197.4366  185.11545  66.086250  


### 4. Generate Predictions using OpenAI API
Use the OpenAI API to generate predictions based on the indicators.

In [11]:
#openai.api_key = 'YOUR_OPENAI_API_KEY'

def predict_market(data):
    responses = []
    for index, row in data.iterrows():
        prompt = f"Date: {index.date()}\n"
        prompt += f"Close: {row['Close']}\n"
        prompt += f"MA50: {row['MA50']}\n"
        prompt += f"MA200: {row['MA200']}\n"
        prompt += f"RSI: {row['RSI']}\n"
        prompt += "Will the market go up or down tomorrow?\n"

        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a financial analyst."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            n=1,
            stop=None,
            temperature=0.7
        )
        prediction = response.choices[0].message.content.strip()
        responses.append(prediction)

    data['Prediction'] = responses
    return data

predictions = predict_market(data)
print(predictions.tail())

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-07-02  216.149994  220.380005  215.100006  220.270004  220.270004   
2024-07-03  220.000000  221.550003  219.029999  221.550003  221.550003   
2024-07-05  221.649994  226.449997  221.649994  226.339996  226.339996   
2024-07-08  227.089996  227.850006  223.250000  227.820007  227.820007   
2024-07-09  227.929993  229.399994  226.369995  228.679993  228.679993   

              Volume      MA50      MA200        RSI  \
Date                                                   
2024-07-02  58046200  192.7818  184.13120  65.641398   
2024-07-03  37369800  193.8960  184.36390  61.367286   
2024-07-05  60412400  195.0848  184.60575  64.784936   
2024-07-08  59085900  196.2608  184.84950  68.856087   
2024-07-09  48076100  197.4366  185.11545  66.086250   

                                                   Prediction  
Date                    

C:\Users\nate.lo\AppData\Local\Temp\ipykernel_17080\4220305234.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Prediction'] = responses


In [55]:
#openai.api_key = 'YOUR_OPENAI_API_KEY'

def predict_market(data):
    historical_data = ""
    for index, row in data.tail(30).iterrows():  # Use the last 30 days of data for context
        historical_data += f"Date: {index.date()}, Close: {row['Close']}, MA50: {row['MA50']}, MA200: {row['MA200']}, RSI: {row['RSI']}\n"
    
    prompt = (
        "Below is the historical market data for the past 30 days:\n"
        f"{historical_data}\n"
        "Based on this data, please predict the market price for the next 7 trading days starting from tomorrow. Please list out the prediction in the format of the provided data above, include columns high low and close.\n"
    )

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a financial analyst."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,  # Increase token limit to allow for longer responses
        n=1,
        stop=None,
        temperature=0.7
    )

    prediction = response.choices[0].message.content.strip()
    return prediction

predictions = predict_market(data)
print(predictions)  # Print the predictions

Based on the historical market data provided, along with the moving averages and relative strength index (RSI) values, I can provide a prediction for the market price for the next 7 trading days starting from tomorrow.

Here is the prediction for the next 7 trading days:

Date: 2024-07-10, Close: 230.00, MA50: 198.6233999633789, MA200: 185.3746000289917, RSI: 68.20087132915326
Date: 2024-07-11, Close: 232.50, MA50: 199.8699996948242, MA200: 185.65530002593995, RSI: 70.59468642643812
Date: 2024-07-12, Close: 235.00, MA50: 201.19459976196288, MA200: 185.9567000389099, RSI: 72.88960045921137
Date: 2024-07-15, Close: 238.00, MA50: 202.59019973754883, MA200: 186.27965000152588, RSI: 75.08443208594729
Date: 2024-07-16, Close: 240.00, MA50: 204.05219970703126, MA200: 186.6242500305176, RSI: 77.1791687976658
Date: 2024-07-17, Close: 242.50, MA50: 205.5741996765137, MA200: 186.9907000732422, RSI: 79.17381557611653
Date: 2024-07-18, Close: 245.00, MA50: 207.1493997192383, MA200: 187.378500061035

### Explanation:
- Data Fetching: Uses yfinance to download historical market data for a specified ticker.
- Indicator Calculation: Computes technical indicators like Moving Averages and RSI.
- Prediction Generation: Sends prompts to the OpenAI API, requesting predictions for market movements.

### Note
- The example uses davinci as the model. Adjust it to gpt-4 or other models if available and suitable.
- Ensure your API key is securely managed.
- The model's responses should be evaluated carefully; this is a simplified approach and might require fine-tuning and validation.

This script provides a basic framework. For a production-level application, consider more advanced data preprocessing, additional indicators, and model validation techniques.